In [113]:
import os
import functools

import tensorflow as tf
from tensorflow.python.ops.signal import shape_ops
import numpy as np

def round_even(x):
    if x % 2 == 0:
        return x + 1
    else:
        return x + 2
    
class ConvRegressor(tf.keras.Model):
    def __init__(
        self, filters, kernel_size, 
        conv_type=tf.keras.layers.Conv2D, activation=tf.nn.relu, 
        **kwargs
    ):
        super().__init__()
        self.filters = filters
        self.conv_type = conv_type
        self.convs = []
        for n_filter in filters:
            self.convs.append(
                conv_type(n_filter, round_even(kernel_size), strides=2, padding='SAME', activation=activation, **kwargs)
            )
            self.convs.append(conv_type(n_filter, kernel_size, padding='SAME', activation=activation, **kwargs))
        self.convs.append(tf.keras.layers.Conv2D(n_filter, kernel_size, padding='SAME'))
    
    @tf.function
    def call(self, x, training=False):
        y = x
        for conv in self.convs:
            y = conv(y, training=training)
        return y

class LogmelSpectrumLayer(tf.keras.layers.Layer):
    def __init__(self, n_mels=128, window_size=2048, hop_size=512, n_fft=2048, window=None, sampling_rate=44.1e3, fmin=0.0, fmax=22050, pad_end=False):
        super().__init__(dynamic=True)
        self.n_mels = n_mels
        self.window_size = window_size
        self.hop_size = hop_size
        self.n_fft = n_fft
        self.pad_end = pad_end

        self.sampling_rate = sampling_rate 
        self.fmin = fmin
        self.fmax = fmax

        if window is None:
            self.window = functools.partial(tf.signal.hann_window, periodic=False)
        else:
            self.window = window

    def compute_output_shape(self, input_shape):
        input_shape = tf.TensorShape(input_shape).as_list()
        frame_axis = input_shape[1]
        frame_step = self.hop_size
        frame_length = self.window_size
        num_frames = None
        
        
        if frame_step is not None and self.pad_end:
            # Double negative is so that we round up.
            num_frames = max(0, -(-frame_axis // frame_step))
        elif frame_step is not None and frame_length is not None:
            num_frames = max(
              0, (frame_axis - frame_length + frame_step) // frame_step)
        return tf.TensorShape([input_shape[0], num_frames, self.n_mels, input_shape[-1]])
    
    def get_config(self):
        return dict(
            n_mels=self.n_mels.numpy(), 
            window_size=self.window_size.numpy(), 
            hop_size=self.hop_size.numpy(), 
            n_fft=self.n_fft.numpy(), 
            window=self.window.__name__ if callable(self.window) else self.window,
            sampling_rate=self.sampling_rate.numpy(), 
            fmin=self.fmin.numpy(), fmax=self.fmax.numpy(),
            pad_end=self.pad_end.numpy()
        )
    
    def call(self, xs, training=False):
        n_batch, _, n_channel = tf.shape(xs)
        xs = tf.reshape(
            tf.transpose(xs, (0, 2, 1)), 
            (-1, tf.shape(xs)[1])
        )
        stfts = tf.signal.stft(
            xs, frame_length=self.window_size,
            frame_step=self.hop_size,
            fft_length=self.n_fft,
            pad_end=False
        )
        spectrum = tf.square(tf.abs(stfts))
        mel_bank = tf.signal.linear_to_mel_weight_matrix(
            self.n_mels, self.n_fft // 2 + 1, 
            self.sampling_rate, self.fmin, self.fmax
        )

        mel_spectrum = tf.math.log(
            tf.tensordot(spectrum, mel_bank, 1) + 1e-8
        )
        
        _, h, w = tf.shape(mel_spectrum)
        return tf.transpose(
            tf.reshape(mel_spectrum, (n_batch, n_channel, h, w)),
            (0, 2, 3, 1)
        )
    
class WavegramCNN(tf.keras.Model):
    def __init__(self, base_net, n_wavegram_ch=3, kernel_size=7, gate_kernel_size=11, strides=3, gate_strides=5, activation=tf.nn.relu, **logmel_kwargs):
        super().__init__(dynamic=True)
        n_mels = logmel_kwargs.get('n_mels', 128)
        self.n_wavegram_ch = n_wavegram_ch
        
        self.wavegram_net = tf.keras.Sequential([
            tf.keras.layers.Conv1D(n_mels // 8, kernel_size=gate_kernel_size, strides=gate_strides, activation=activation),
            
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Conv1D(n_mels // 4, kernel_size=kernel_size, dilation_rate=2, strides=1, activation=activation),
            tf.keras.layers.Conv1D(n_mels // 4, kernel_size=kernel_size, dilation_rate=1, strides=strides, activation=activation),

            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Conv1D(n_mels // 2, kernel_size=kernel_size, dilation_rate=2, strides=1, activation=activation),
            tf.keras.layers.Conv1D(n_mels // 2, kernel_size=kernel_size, dilation_rate=1, strides=strides, activation=activation),

            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Conv1D(n_mels, kernel_size=kernel_size, dilation_rate=2, strides=1, activation=activation),
            tf.keras.layers.Conv1D(n_mels, kernel_size=kernel_size, dilation_rate=1, strides=strides, activation=activation),\
        ])
        self.logmel_layer = LogmelSpectrumLayer(**logmel_kwargs)
        self.cnn = base_net
    
#     @tf.function
    def call(self, x, training=False):
#         x = x / tf.reduce_max(x, axis=(1, 2), keepdims=True)
        logmel  = self.logmel_layer(x, training=training)
        wavegram = self.wavegram_net(x, training=training)
        
        batch_size = tf.shape(x)[0]
        n_frames = tf.shape(logmel)[1]
        n_filters = tf.shape(wavegram)[1]
        n_wavegram_dim = n_frames * self.n_wavegram_ch
        wavegram_offset = (n_filters - n_wavegram_dim) // 2
        
        wavegram = tf.transpose(
            tf.reshape(
                wavegram[:, wavegram_offset:wavegram_offset+n_wavegram_dim, :],
                (batch_size, n_frames, self.n_wavegram_ch, -1) # N, T, C, F
            ), (0, 1, 3, 2) # N, T, F, C
        )

        return self.cnn(tf.concat([wavegram, logmel], axis=-1), training=training)

layer = WavegramCNN(tf.keras.Sequential([tf.keras.layers.Flatten(), tf.keras.layers.Dense(10)]))
t = layer(tf.zeros((4, 32000, 2)))
# print(layer.compute_output_shape((4, 32000, 2)))
print(t)


tf.Tensor(
[[ 12.910117    1.2321125 -18.948282   26.127281   -2.4666774 -22.519981
    6.820009   17.862968   -5.7939076   7.2299757]
 [ 12.910117    1.2321125 -18.948282   26.127281   -2.4666774 -22.519981
    6.820009   17.862968   -5.7939076   7.2299757]
 [ 12.910117    1.2321125 -18.948282   26.127281   -2.4666774 -22.519981
    6.820009   17.862968   -5.7939076   7.2299757]
 [ 12.910117    1.2321125 -18.948282   26.127281   -2.4666774 -22.519981
    6.820009   17.862968   -5.7939076   7.2299757]], shape=(4, 10), dtype=float32)


In [114]:
import importlib
from libs.layers import resnet
from libs.layers import cbam_resnet
from libs.layers import conv_attention_resnet

ResNet = resnet.ResNet34
ResNetCBAM = cbam_resnet.ResNetCBAM34
AttentionResNet = conv_attention_resnet.AttentionResNet34
AxialAttentionResNet = conv_attention_resnet.AxialAttentionResNet34

In [115]:
fold_index = 0
task = 'home'
model_type = 'attention'

model_dir = './model'
data_dir = './data'



In [116]:
# tf.config.experimental.set_memory_growth(
#     tf.config.list_physical_devices('GPU')[0], allow_memory_growth
# )

In [117]:
from pathlib import Path
model_dir = Path(model_dir)
data_dir = Path(data_dir)
model_dir.mkdir(exist_ok=True, parents=True)

In [118]:
# Load training setup
import pandas as pd
from pathlib import Path
from libs.misc import wavio
from tqdm.notebook import tqdm

root_dir = data_dir / 'evaluation_setup'

data_cache = {}

def get_annotation(task, fold_index, target):
    df = pd.read_csv(
        root_dir / f'{task}_fold{fold_index+1}_{target}.txt', sep='\t', 
        header=None, names=['file', 'class', 'start', 'end', 'event']
    )
    df['id'] = df['file'].apply(lambda x: Path(x).stem)
    return df

def load_dataset(target):
    if target not in data_cache:
        df = get_annotation(task, fold_index, target)
        wav_dict = {}
        for file in tqdm(df['file'].unique()):
            wav_dict[Path(file).stem] = wavio.readwav(str(data_dir / file))
        data_cache[target] = (df, wav_dict)
        return df, wav_dict
    else:
        return data_cache[target]
df, wav_dict = load_dataset('train')

In [199]:
from itertools import product
import numpy as np

sound_events = ['nan']

for fold, task_name in product([0,1,2,3], ['train', 'evaluate', 'test']):
    sound_events.extend(get_annotation('home', fold, task_name)['event'].unique())
sound_events = list(filter(lambda x: x != 'nan', np.unique(sound_events)))
sound_events = ['nan'] + sound_events
print(sound_events)

['nan', '(object) rustling', '(object) snapping', 'cupboard', 'cutlery', 'dishes', 'drawer', 'glass jingling', 'object impact', 'people walking', 'washing dishes', 'water tap running']


In [200]:
from functools import partial

n_augmentation = 5
perturbation = (0.0, 0.05)

def parse_wave(series, wav_dict, start=0.0, end=5.0, duration=5.0):    
    result = series.to_dict()
    sr, bw, audio = wav_dict[series['id']]
    
    data = np.zeros((int(duration * sr), 2), dtype=np.float32)
    mask = np.zeros((int(duration * sr), 1), dtype=np.bool)
    
    start = int(start * sr)
    end = int(end * sr)
    data[start:end] = audio[start:end]
    mask[start:end] = True
    
    result['id'] = series['id']
    result['start_index'] = max(start, 0)
    result['end_index'] = min(end, len(audio))
    result['sr'] = sr
    result['bw'] = bw
    result['audio'] = data
    result['mask'] = mask
    
    return pd.Series(result)

_id = df.loc[0, 'id']
sr = wav_dict[_id][0]
# display(Audio(wav_dict[_id][2][:, 0], rate=sr))
# display(Audio(
#     parse_wave(df.loc[0], wav_dict=wav_dict)['audio'][:, 0], rate=sr
# ))
# audio_df = df.apply(partial(parse_wave, wav_dict=wav_dict, window_size=5.0, hop_size=1.25), axis=1)

In [201]:
cache = {}

In [202]:
from functools import partial
import librosa
import joblib
import random 

n_mels = 128
n_sampling=4096
hop_length = n_sampling // 4
window='hann'
pad='constant'
max_time = 1.0
batch_size = 2

def preprocess(wav, sampling_rate):
    return np.concatenate([
        mono_preprocess(wav[..., 0], sampling_rate)[..., np.newaxis],
        mono_preprocess(wav[..., 1], sampling_rate)[..., np.newaxis],
    ], axis=-1)

def mono_preprocess(wav, sampling_rate):
    mag = librosa.feature.melspectrogram(
        wav, sr=sampling_rate, hop_length=hop_length, n_mels=n_mels,
        fmin=0.0, fmax=20e3,
    )
    logmag = np.log(mag + 1e8)
    return logmag

def normalize_time(audio_df, max_time):
    results = []
    for audio, sr in zip(audio_df['audio'], audio_df['sr']):
        max_len = int(sr * max_time)

        pos = min(len(audio), max_len)
        result = np.zeros((max_len, 2), np.float32)
        result[:pos, :] = audio[:pos]
        results.append(result)
    return results


def get_dataset(target):
    df, wav_dict = load_dataset(target)
    encoding_dict = {event: i for i, event in enumerate(sound_events)}
    
    def yield_wave():
        def _parse_wave(series):
            start = random.uniform(0.0, 0.1 * max_time)
            end = random.uniform(0.1 * max_time, max_time)
            end = end if (max_time - start) > (start + end) else (max_time - start)
            return parse_wave(series, wav_dict, start=start, end=end, duration=max_time)
            
        for i in df.index:
            series = _parse_wave(df.loc[i])
            event = encoding_dict[series['event']]
            yield series['audio'], series['mask'], event
    return yield_wave
    
def get_tf_dataset(target, batch_size=32, shuffle=False, **kwargs):
    ds = tf.data.Dataset.from_generator(
        get_dataset(target, **kwargs),
        output_types=(tf.float32, tf.bool, tf.int32)
    )
    if shuffle:
        ds = ds.shuffle(buffer_size=1000, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size, drop_remainder=shuffle)
    return ds

In [203]:
train_dataset = get_tf_dataset('train', batch_size=batch_size, shuffle=True)
test_dataset = get_tf_dataset('evaluate', batch_size=batch_size)

In [204]:
from collections import defaultdict

class PlotCallback(tf.keras.callbacks.Callback):
    is_higher_better = {
        'accuracy'
    }
    is_linear = {
        'accuracy',
        'sparse_categorical_accuracy',
    }
    def __init__(self, targets=None, n_step=1):
        super().__init__()
        self.fig = None
        self.axes = None
        self.axes_index = {}
        self.n_step = n_step
        self.targets = targets
        self.epochs = []
        self.history = defaultdict(list)

    def plot_and_display(self):
        for ax in self.axes.flat:
            ax.clear()
        for i, (label, values) in enumerate(self.history.items()):
            if any(name in label for name in self.is_higher_better):
                get_best_value = np.amax
            else:
                get_best_value = np.amin
            
            if label.startswith('val_'):
                _label = label[4:]
            else:
                _label = label
            
            ax = self.axes.flat[self.axes_index[_label]]
            ax.plot(self.epochs, values, label=label, color=f'C{i}')
            best_value = get_best_value(values)
            ax.axhline(best_value, linestyle='--', color=f'C{i}')
            ax.text(0.0, best_value, f'{best_value:.3f}')
            
            if _label not in self.is_linear:
                ax.set_yscale('log')

        if self.epochs[-1] == 0:
            self.fig.legend()

        io = BytesIO()
        self.fig.savefig(io, format='png')

        clear_output(wait=True)
        display_png(Image(io.getvalue()))
        
    def on_epoch_end(self, epoch, logs):
        if epoch == 0:
            self.fig, self.axes = plt.subplots(len(logs) // 2, 1, figsize=(8, 4 * len(logs) // 2))
            self.axes_index = {}
            for label in logs:
                if label.startswith('val_'):
                    _label = label[4:]
                else:
                    _label = label
                if _label not in self.axes_index:
                    self.axes_index[_label] = len(self.axes_index)

        for key, value in logs.items():
            self.history[key].append(value)

        self.epochs.append(epoch)
        if (epoch % self.n_step) == 0:
            self.plot_and_display()

class BalancedSparseCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):
    def __init__(self, name='balanced_sparse_categorical_accuracy', dtype=None):
        super().__init__(name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_flat = y_true
        if y_true.shape.ndims == y_pred.shape.ndims:
            y_flat = tf.squeeze(y_flat, axis=[-1])
        y_true_int = tf.cast(y_flat, tf.int32)

        cls_counts = tf.math.bincount(y_true_int)
        cls_counts = tf.math.reciprocal_no_nan(tf.cast(cls_counts, self.dtype))
        weight = tf.gather(cls_counts, y_true_int)
        return super().update_state(y_true, y_pred, sample_weight=weight)
    
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [221]:
def create_cnn_model():
    return tf.keras.Sequential([
        ConvRegressor([8, 16, 32, 64, ], kernel_size=7),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(sound_events)),
    ])

def create_resnet_cnn_model():
    return tf.keras.Sequential([
        ResNet(kernel_size=5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(sound_events)),
    ])

def create_cbam_model():
    return tf.keras.Sequential([
        ResNetCBAM(kernel_size=5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(sound_events)),
    ])

def create_attention_model():
    return tf.keras.Sequential([
        AttentionResNet(kernel_size=1),
        tf.keras.layers.Conv2D(1024, kernel_size=7, strides=2, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(1024, kernel_size=7, strides=2, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(1024, kernel_size=7, strides=2, activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(sound_events)),
    ])

def create_axial_attention_model():
    return tf.keras.Sequential([
        AxialAttentionResNet(kernel_size=1),
        tf.keras.layers.Conv2D(1024, kernel_size=7, strides=2, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(1024, kernel_size=7, strides=2, activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(sound_events)),
    ])

model_type = 'cnn'
if model_type == 'cnn':
    learning_rate = 1e-5
    create_model = create_cnn_model
elif model_type == 'resnet':
    learning_rate = 1e-5
    create_model = create_resnet_cnn_model
elif model_type == 'cbam':
    learning_rate = 1e-4
    create_model = create_cbam_model
elif model_type == 'attention':
    create_model = create_attention_model
    learning_rate = CustomSchedule(128)
elif model_type == 'axial-attention':
    learning_rate = 1e-4
    create_model = create_axial_attention_model
else:
    raise NotImplementedError()


In [222]:
# import imageio
# import librosa.display
# import matplotlib.pyplot as plt
# from scipy.io import wavfile 
# from pathlib import Path

# output_dir = Path('audio')
# output_dir.mkdir(exist_ok=True)

# logmel_layer = LogmelSpectrumLayer()
# for i, (audio, mask, event) in tqdm(enumerate(train_dataset.unbatch().as_numpy_iterator())):
#     logmel_spec = logmel_layer(tf.expand_dims(audio, axis=0)).numpy()
    
#     fig, axes = plt.subplots(1, 2, figsize=(12, 5))
#     for j in range(2):
#         ax = axes.flat[j]
#         mappable = librosa.display.specshow(
#             logmel_spec[0, ..., j].T,
#             sr=44100, hop_length=512,
#             x_axis='time', y_axis='mel',
#             fmin=0.0, fmax=20.0e3,
#             ax=ax, # cmap='inferno'
#         )
#         mappable.set_clim(-35.0, 35.0)
#     colorbar = fig.colorbar(mappable)
#     plt.close(fig)
#     fig.savefig(str(output_dir / f'{i:04d}_{sound_events[event]}.png'))
#     wavfile.write(str(output_dir / f'{i:04d}_{sound_events[event]}.wav'), 44100, audio.astype(np.int16))

In [ ]:
import IPython 
from collections import defaultdict 
from itertools import product
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import time
import seaborn as sns
from io import BytesIO
import imageio
from IPython.display import Image, display_png, clear_output

#%matplotlib widget
%matplotlib inline
#%matplotlib notebook
if 'model' in globals():
    del model
epochs = 2000
base_net = create_model()
base_net.add(tf.keras.layers.Activation('softmax'))
model = WavegramCNN(base_net)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5, beta_2=0.99)

checkpoint_name = f'{task}_cnn_{model_type}_fold{fold_index}'
cur_model_dir = model_dir / checkpoint_name
cur_model_dir.mkdir(exist_ok=True, parents=True)

plot_callback = PlotCallback(n_step=3)
with tf.device('/GPU:0'):
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False, reduction=tf.keras.losses.Reduction.SUM),
        optimizer=optimizer,
        metrics=BalancedSparseCategoricalAccuracy(),
#         options=tf.distribute.RunOptions(report_tensor_allocations_upon_oom = True)
    )
    
    mode = 'min'
    model.fit(
        train_dataset.map(lambda *vars_list: (vars_list[0], vars_list[-1])),
        batch_size=batch_size, epochs=2000, shuffle=True,
        validation_data=test_dataset.map(lambda *vars_list: (vars_list[0], vars_list[-1])),
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                patience=50, 
                monitor='val_loss',
                #monitor='val_balanced_sparse_categorical_accuracy',
                mode=mode
            ),
            plot_callback,
            tf.keras.callbacks.TerminateOnNaN(),
            tf.keras.callbacks.ModelCheckpoint(
                str(cur_model_dir / (checkpoint_name + '.model')),
                monitor='val_loss',
                #monitor='val_balanced_sparse_categorical_accuracy', 
                save_best_only=True,
                save_weights_only=True,
                mode=mode, 
            )
        ]
    )

plot_callback.fig.tight_layout()
plot_callback.fig.savefig(str(cur_model_dir / (checkpoint_name + '.png')))

Epoch 1/2000
    155/Unknown - 5s 35ms/step - loss: 4.7111 - balanced_sparse_categorical_accuracy: 0.2301

In [ ]:
from itertools import product
from sklearn.metrics import accuracy_score
accuracy_data = []
model = create_model()
model.load_weights(str(cur_model_dir / (checkpoint_name + '.model')))
model.compile()

results = {}
for target_name, dataset in zip(['train', 'test'], (train_dataset, test_dataset)):
    pred_logits = model.predict(
        dataset.map(lambda audios, norm_audios, labels: (audios, labels))
    )
    pred_labels = tf.argmax(tf.nn.softmax(pred_logits, axis=1), axis=1)
    
    audio = []
    truth_labels = []
    for batch in dataset:
        audio.extend(batch[1].numpy())
        truth_labels.extend(batch[2].numpy())
    metric = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(
        tf.convert_to_tensor(np.array(truth_labels).astype(np.int32)), 
        tf.convert_to_tensor(pred_logits.astype(np.float32))
    )).numpy()

    truth_labels = np.array(sound_events).take(truth_labels)
    pred_labels = np.array(sound_events).take(pred_labels)
    
    agg_df = pd.crosstab(
        pd.Series(truth_labels, name='Truth'),
        pd.Series(pred_labels, name='Prediction'),
    )
    agg_df = agg_df.reindex(columns=sound_events, index=sound_events, fill_value=0)
    display(target_name)
    display(agg_df)
    
    accuracy = {}
    for name in sound_events:
        mask = truth_labels == name
        accuracy[name] = accuracy_score(truth_labels[mask], pred_labels[mask]) 
    accuracy['Metric'] = metric
    accuracy_data.append(pd.Series(accuracy, name=target_name))
    
    results[target_name] = {
        'Audio': None if save_without_train and target_name == 'train' else audio,
        'Prediction': pd.DataFrame({
            'Prediction': pred_labels,
            'Truth': truth_labels,
        }),
        'Agg': agg_df,
        'Accuracy': accuracy_data,
    }
accuracy_df = pd.DataFrame(accuracy_data)
accuracy_df['Mean'] = accuracy_df.mean(axis=1)
display(accuracy_df.T)

In [ ]:
import cloudpickle as pickle
with open(cur_model_dir / f'result_metric.pickle', 'wb+') as fp:
    pickle.dump(results, fp)

In [75]:
save_without_train and 'train' == 'train'

True